# Creating colorized point clouds for bridges

This lab will create and colorize point clouds for three bridges in your study area and calculate their height above the stream. You will use these point clouds (and thier derivatives) to measure the physical properties of these bridges. If we find that the calculated bridge height is close to the measured height, we will use part of this analysis on all bridges.

## Instructions

1. Create a new ArcGIS Pro **Map** Project with a name that does not contain spaces.
2. Verify your project name and that the project contains a geodatabase using the project name.
3. Add you bridges layer you created in the first lab. Select three bridges; one bridge that was in the original data, one that you found and digitized, and one that you suspect is a typical private bridge. Export those three bridges to the geodatabase that was created in Step 1.
4. Modify the cell below that sets the variables for your local environment. You should not need to modify any other cell.
5. Run the remaining cells in order, top down. If you get errors, check your variables again. Rerun that cell and continue.

In [ ]:
# Import ArcGIS package
import arcpy
# Subprocess allows us to issue laszip64.exe commands on the command line
import subprocess

In [ ]:
######### MODIFY THIS CELL: change to variables to match your local environment ######### 

# Where is your root GIS folder?
r = 'c:\\BoydsGIS'

# What is your project's folder name?
p = 'three_bridges'

# What is your bridges layer name?
b = 'example_bridges'

# Where is the laszip64.exe located on your computer?
e = 'c:\\BoydsGIS\\data\\laszip64.exe'

# Where is the KyFromAbove_Data_Product_Tile_Grids geodatabase located on your computer?
t = 'c:\\BoydsGIS\\data\\KyFromAbove_Data_Product_Tile_Grids.gdb'

# How far do you want to extract point cloud data from your bridges?
# This distance is in feet.
buffer = 500

In [ ]:
# Set up our environment
arcpy.env.workspace = f'{r}\\{p}\\{p}.gdb'
ky = arcpy.SpatialReference(3089)
arcpy.env.outputCoordinateSystem = ky
arcpy.env.overwriteOutput = True
pd = f'{r}\\{p}\\download\\'
pl = f'{r}\\{p}\\las\\'
pc = f'{r}\\{p}\\colorize\\'
subprocess.run(f'mkdir {pd}', shell=True)
subprocess.run(f'mkdir {pl}', shell=True)
subprocess.run(f'mkdir {pc}', shell=True)
ro = f'{r}\\{p}'
bo = f'brdg_buf_{buffer}ft'
bands = 'RED Band_1; GREEN Band_2; BLUE Band_3'
img = 'https://kyraster.ky.gov/arcgis/rest/services/ImageServices/Ky_KYAPED_Phase1_6IN/ImageServer'
nmg = 'https://kyraster.ky.gov/arcgis/rest/services/ImageServices/Ky_NAIP_2012_1M/ImageServer'
dsm = 'https://kyraster.ky.gov/arcgis/rest/services/ElevationServices/Ky_DSM_First_Return_5FT_Phase1/ImageServer'
dem = 'https://kyraster.ky.gov/arcgis/rest/services/ElevationServices/Ky_DEM_KYAPED_5FT/ImageServer'
print(arcpy.ListFeatureClasses())

In [ ]:
# Buffer the bridges lauer
arcpy.analysis.Buffer(b, bo, buffer)

In [ ]:
# Find the tiles that we need to download
arcpy.env.addOutputsToMap = False
selection = arcpy.management.SelectLayerByLocation(f'{t}\\Kentucky_5k_PointCloudGrid', "INTERSECT", bo)
arcpy.env.addOutputsToMap = True
arcpy.conversion.ExportFeatures(selection, 'tiles_to_download')

In [ ]:
# Decompress our lAZ files and assemble the LAS dataset
las = []
subprocess.run(f'dir {pd}\\*.laz /b > {pd}\\zips.txt', shell=True, stdout=subprocess.PIPE)
with open(f'{pd}\\zips.txt', 'r') as fileList:
    for line in fileList:
        name = line.strip()
        subprocess.run(f'{e} -i {pd}\\{name} -o {pl}\\{name[:-4]}.las', shell=True, stdout=subprocess.PIPE)
        las.append(f'{pl}\\{name[:-4]}.las')
arcpy.CreateLasDataset_management (las, f'{ro}\\aoi.lasd', "#", "#", ky, True, True)

In [ ]:
# Iterate over each bridge AOI and do analysis
arcpy.env.addOutputsToMap = False
lasd = f'{ro}\\aoi.lasd'
with arcpy.da.SearchCursor(bo,['SHAPE@','OBJECTID']) as cursor:
    for row in cursor:
        r0 = row[0]
        r1 = row[1]
        print(f'Processing AOI {r1}...')
        eaoi = f'{pl}aoi_{r1}'
        caoi = f'{pc}aoi_{r1}'
        subprocess.run(f'mkdir {eaoi}', shell=True)
        subprocess.run(f'mkdir {caoi}', shell=True)
        subprocess.run(f'del /q {caoi}\\*.*', shell=True)
        print(f'Extracting point cloud...')
        arcpy.ExtractLas_3d (lasd, eaoi, "#", r0,  "#", "_e", "#", "#", True, f'{ro}\\aoi_{r1}.lasd')
        print(f'Clipping aerial photography...')
        arcpy.management.Clip(img, '#', f'img_6in_aoi_{row[1]}', row[0], '#', True)
        print(f'Colorizing point cloud...')
        arcpy.ColorizeLas_3d (f'{ro}\\aoi_{r1}.lasd', f'img_6in_aoi_{r1}', bands, caoi, "_rgb", "#", "#","#", "#", True, f'{ro}\\aoi_{r1}_rgb.lasd')
        print(f'Clipping DSM...')
        arcpy.management.Clip(dsm, '#', f'dsm_5ft_aoi_{r1}', r0, '#', True)
        arcpy.HillShade_3d(f'dsm_5ft_aoi_{r1}', f'hillshade_dsm_aoi_{r1}')
        print(f'Clipping DEM...')
        arcpy.management.Clip(dem, '#', f'dem_5ft_aoi_{r1}', r0, '#', True)
        print(f'Calculating height of above-ground features...')
        minus = arcpy.sa.Minus(f'dsm_5ft_aoi_{r1}',f'dem_5ft_aoi_{r1}')
        minus.save(f'height_aoi_{r1}')
        print(f'Adding height to {b} layer as field "Z" in feet above stream...')
        selection = arcpy.management.SelectLayerByAttribute(b, '#', f"OBJECTID = {r1}")
        arcpy.sa.AddSurfaceInformation(selection, minus, 'Z')
        print(f'Removing {ro}\\aoi_{r1}.lasd and {eaoi}')
        subprocess.run(f'del {ro}\\aoi_{r1}.lasd', shell=True)
        subprocess.run(f'rmdir /s /q {eaoi}', shell=True)
        print(f'Processing complete for AOI {r1}\n')

print('Finished!')
arcpy.env.addOutputsToMap = True